In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/jdariasl/DiplomadoEPM_2019/master/Modulo3/init.py
from init import init; init(force_download=False)

In [1]:
try:
    %tensorflow_version 2.x
    print ("Using TF2 in Google Colab")
except:
    pass
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from local.lib import mlutils
%matplotlib inline
from sklearn.datasets import *
tf.__version__

'2.1.0'

# LAB: add embeddings and cross columns


### check first [this tutorial](https://www.tensorflow.org/tutorials/structured_data/feature_columns)

**this is the same code as in the corresponding notebook**

In [2]:
import pandas as pd

In [3]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 
        'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
        'label' ]

In [4]:
d = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                names=cols)

In [5]:
train_pct = .7
p = np.random.permutation(len(d))
idxs_train = p[:int(len(p)*train_pct)]
idxs_test  = p[int(len(p)*train_pct):]
len(d), len(idxs_train), len(idxs_test)

(32561, 22792, 9769)

In [6]:
labels = np.r_[[0 if "<=50K" in i else 1 for i in d.label]]

train_ds = tf.data.Dataset.from_tensor_slices( 
    (dict(d.iloc[idxs_train][[i for i in d.columns if i!='label']]), labels[idxs_train]))

test_ds = tf.data.Dataset.from_tensor_slices( 
    (dict(d.iloc[idxs_test][[i for i in d.columns if i!='label']]), labels[idxs_test]))

In [7]:
fage  = tf.feature_column.numeric_column('age', default_value=0)
feducation_num = tf.feature_column.numeric_column("education-num", default_value=0)
fcapital_gain  = tf.feature_column.numeric_column("capital-gain", default_value=0)
fcapital_loss  = tf.feature_column.numeric_column("capital-loss", default_value=0)
hpw            = tf.feature_column.numeric_column("hours-per-week", default_value=0)

frace = tf.feature_column.categorical_column_with_vocabulary_list("race", list(np.unique(d.race)))
crace = tf.feature_column.indicator_column(frace)

fmar = tf.feature_column.categorical_column_with_vocabulary_list("marital-status", list(np.unique(d["marital-status"])))
cmar = tf.feature_column.indicator_column(fmar)

focc = tf.feature_column.categorical_column_with_vocabulary_list("occupation", list(np.unique(d["occupation"])))
cocc = tf.feature_column.indicator_column(focc)


In [8]:
def get_model(featureset):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.DenseFeatures(featureset, dtype=tf.float64))
    model.add(tf.keras.layers.Dense(100, activation="tanh"))
    model.add(tf.keras.layers.Dense(50, activation="tanh"))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam",
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=['accuracy'])
    return model

### TASK 1: add an embedding column for `native-country` with dimension 10

- observe that your **FIRST** have to create a categorical column and **THEN** use it to create the embedding column
- use **ALL** available features.

In [9]:
ecountry = ...
features = ...
model = ...
model.fit(...)

In [16]:
# uncomment these lines to see the solution
#sol = [10, 102, 99, 111, 117, 110, 116, 114, 121, 32, 61, 32, 116, 102, 46, 102, 101, 97, 116, 117, 114, 101, 95, 99, 111, 108, 117, 109, 110, 46, 99, 97, 116, 101, 103, 111, 114, 105, 99, 97, 108, 95, 99, 111, 108, 117, 109, 110, 95, 119, 105, 116, 104, 95, 118, 111, 99, 97, 98, 117, 108, 97, 114, 121, 95, 108, 105, 115, 116, 40, 34, 110, 97, 116, 105, 118, 101, 45, 99, 111, 117, 110, 116, 114, 121, 34, 44, 32, 108, 105, 115, 116, 40, 110, 112, 46, 117, 110, 105, 113, 117, 101, 40, 100, 91, 34, 110, 97, 116, 105, 118, 101, 45, 99, 111, 117, 110, 116, 114, 121, 34, 93, 41, 41, 41, 10, 101, 99, 111, 117, 110, 116, 114, 121, 32, 61, 32, 116, 102, 46, 102, 101, 97, 116, 117, 114, 101, 95, 99, 111, 108, 117, 109, 110, 46, 101, 109, 98, 101, 100, 100, 105, 110, 103, 95, 99, 111, 108, 117, 109, 110, 40, 102, 99, 111, 117, 110, 116, 114, 121, 44, 32, 100, 105, 109, 101, 110, 115, 105, 111, 110, 61, 49, 48, 41, 10, 102, 101, 97, 116, 117, 114, 101, 115, 32, 61, 32, 91, 102, 97, 103, 101, 44, 32, 102, 101, 100, 117, 99, 97, 116, 105, 111, 110, 95, 110, 117, 109, 44, 32, 102, 99, 97, 112, 105, 116, 97, 108, 95, 103, 97, 105, 110, 44, 32, 102, 99, 97, 112, 105, 116, 97, 108, 95, 108, 111, 115, 115, 44, 32, 104, 112, 119, 44, 32, 99, 114, 97, 99, 101, 44, 32, 99, 109, 97, 114, 44, 32, 99, 111, 99, 99, 44, 32, 101, 99, 111, 117, 110, 116, 114, 121, 93, 10, 109, 111, 100, 101, 108, 32, 61, 32, 103, 101, 116, 95, 109, 111, 100, 101, 108, 40, 102, 101, 97, 116, 117, 114, 101, 115, 41, 10, 109, 111, 100, 101, 108, 46, 102, 105, 116, 40, 116, 114, 97, 105, 110, 95, 100, 115, 46, 98, 97, 116, 99, 104, 40, 54, 52, 41, 44, 32, 101, 112, 111, 99, 104, 115, 61, 49, 48, 44, 32, 118, 97, 108, 105, 100, 97, 116, 105, 111, 110, 95, 100, 97, 116, 97, 61, 116, 101, 115, 116, 95, 100, 115, 46, 98, 97, 116, 99, 104, 40, 49, 50, 56, 41, 41, 10]
#print ("".join([chr(i) for i in sol]))

### TASK 2: create a cross column between `race` and `occupation` with `hash_bucket_size=100`

observe that:

- **FIRST** you need to create the cross column using the corresponding category columns
- **THEN** you may choose to use it as input to an `indicator_column` or an `embedding_column`

In [ ]:
newcol = ...
features = ...
model = ...
model.fit(...)

In [19]:
# uncomment these lines to see the solution
#sol = [10, 102, 120, 99, 111, 108, 49, 32, 61, 32, 116, 102, 46, 102, 101, 97, 116, 117, 114, 101, 95, 99, 111, 108, 117, 109, 110, 46, 99, 114, 111, 115, 115, 101, 100, 95, 99, 111, 108, 117, 109, 110, 40, 91, 102, 114, 97, 99, 101, 44, 32, 102, 111, 99, 99, 93, 44, 32, 104, 97, 115, 104, 95, 98, 117, 99, 107, 101, 116, 95, 115, 105, 122, 101, 61, 49, 48, 48, 41, 10, 99, 120, 99, 111, 108, 49, 32, 61, 32, 116, 102, 46, 102, 101, 97, 116, 117, 114, 101, 95, 99, 111, 108, 117, 109, 110, 46, 105, 110, 100, 105, 99, 97, 116, 111, 114, 95, 99, 111, 108, 117, 109, 110, 40, 102, 120, 99, 111, 108, 49, 41, 10, 101, 120, 99, 111, 108, 49, 32, 61, 32, 116, 102, 46, 102, 101, 97, 116, 117, 114, 101, 95, 99, 111, 108, 117, 109, 110, 46, 101, 109, 98, 101, 100, 100, 105, 110, 103, 95, 99, 111, 108, 117, 109, 110, 40, 102, 120, 99, 111, 108, 49, 44, 32, 100, 105, 109, 101, 110, 115, 105, 111, 110, 61, 50, 48, 41, 10, 10, 35, 32, 99, 104, 111, 111, 115, 101, 32, 101, 105, 116, 104, 101, 114, 32, 111, 110, 101, 10, 102, 101, 97, 116, 117, 114, 101, 115, 32, 61, 32, 91, 102, 97, 103, 101, 44, 32, 102, 101, 100, 117, 99, 97, 116, 105, 111, 110, 95, 110, 117, 109, 44, 32, 102, 99, 97, 112, 105, 116, 97, 108, 95, 103, 97, 105, 110, 44, 32, 102, 99, 97, 112, 105, 116, 97, 108, 95, 108, 111, 115, 115, 44, 32, 104, 112, 119, 44, 32, 99, 114, 97, 99, 101, 44, 32, 99, 109, 97, 114, 44, 32, 99, 111, 99, 99, 44, 32, 101, 99, 111, 117, 110, 116, 114, 121, 44, 32, 99, 120, 99, 111, 108, 49, 93, 10, 102, 101, 97, 116, 117, 114, 101, 115, 32, 61, 32, 91, 102, 97, 103, 101, 44, 32, 102, 101, 100, 117, 99, 97, 116, 105, 111, 110, 95, 110, 117, 109, 44, 32, 102, 99, 97, 112, 105, 116, 97, 108, 95, 103, 97, 105, 110, 44, 32, 102, 99, 97, 112, 105, 116, 97, 108, 95, 108, 111, 115, 115, 44, 32, 104, 112, 119, 44, 32, 99, 114, 97, 99, 101, 44, 32, 99, 109, 97, 114, 44, 32, 99, 111, 99, 99, 44, 32, 101, 99, 111, 117, 110, 116, 114, 121, 44, 32, 101, 120, 99, 111, 108, 49, 93, 10, 109, 111, 100, 101, 108, 32, 61, 32, 103, 101, 116, 95, 109, 111, 100, 101, 108, 40, 102, 101, 97, 116, 117, 114, 101, 115, 41, 10, 109, 111, 100, 101, 108, 46, 102, 105, 116, 40, 116, 114, 97, 105, 110, 95, 100, 115, 46, 98, 97, 116, 99, 104, 40, 54, 52, 41, 44, 32, 101, 112, 111, 99, 104, 115, 61, 49, 48, 44, 32, 118, 97, 108, 105, 100, 97, 116, 105, 111, 110, 95, 100, 97, 116, 97, 61, 116, 101, 115, 116, 95, 100, 115, 46, 98, 97, 116, 99, 104, 40, 49, 50, 56, 41, 41, 10]
#print ("".join([chr(i) for i in sol]))